pip install tensorflow  


In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import imageio

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
def make_nn():
    L = 9
    kernel_size = 5
    patch_size = 1080
    input_channels = 28
    hidden_channels = 100
    in_between_layers = 7
    output_kernel_size = 28

    model = keras.Sequential()
    model.add(keras.Input(shape=(patch_size, patch_size, input_channels)))
    for i in range(in_between_layers):
        model.add(keras.layers.ZeroPadding2D(padding=(2, 2)))
        model.add(layers.Conv2D(hidden_channels, kernel_size=kernel_size, strides=1, activation="relu"))
    model.add(keras.layers.ZeroPadding2D(padding=(2, 2)))
    model.add(layers.Conv2D(output_kernel_size**2, kernel_size=kernel_size, strides=1, activation="relu"))
    model.compile(loss="mean_absolute_error", optimizer="adam", metrics=['accuracy'])
    return model;
model = make_nn()

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 1084, 1084, 28)    0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1080, 1080, 100)   70100     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 1084, 1084, 100)   0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1080, 1080, 100)   250100    
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 1084, 1084, 100)   0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1080, 1080, 100)   250100    
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 1084, 1084, 100)   0

In [28]:
def preprocess_color(color, albedo, epsilon=0.00316):
    return color * (albedo + epsilon)

def calculate_gradient(image):
    y, x, c = image.shape
    dx = image[:, 1:, :] - image[:, :x-1, :]
    dy = image[1:, :, :] - image[:y-1, :, :]
    dx = np.append(dx, np.zeros([y, 1, c]), axis=1)
    dy = np.append(dy, np.zeros([1, x, c]), axis=0)
    grad = np.dstack((dx, dy))
    return grad

def load_image(folder_path):
    # albedo buffers
    albedo = imageio.imread(folder_path + "/albedo.png")
    albedo_variance = imageio.imread(folder_path + "/albedoVariance.png")
    albedo_gradient = calculate_gradient(albedo)

    # color buffers
    color = imageio.imread(folder_path + "/color.png")
    color = preprocess_color(color, albedo)
    color_variance = imageio.imread(folder_path + "/colorVariance.png")
    color_gradient = calculate_gradient(color)

    # depth buffers
    depth = imageio.imread(folder_path + "/depth.png")
    depth_variance = imageio.imread(folder_path + "/depthVariance.png")
    depth_gradient = calculate_gradient(depth)

    # normal buffers
    normal = imageio.imread(folder_path + "/normal.png")
    normal_variance = imageio.imread(folder_path + "/normalVariance.png")
    normal_gradient = calculate_gradient(normal)


    combined = np.dstack((albedo, albedo_variance, albedo_gradient, color, color_variance, color_gradient, depth, depth_variance, depth_gradient, normal, normal_variance, normal_gradient))
    print(combined.shape)

load_image('./pngs/1/png16/')

ValueError: not enough values to unpack (expected 3, got 2)